In [4]:
import ee
import geemap
import pandas as pd
from ipyleaflet import Popup, Marker

# Initialize Earth Engine
ee.Initialize(project='data690-zhouhaomatt')


# Set display options to show all columns
pd.set_option('display.max_columns', None)


In [15]:
# Define the central point and buffer

initial_point = ee.Geometry.Point(170.88861830313817, -43.99665917291535)
aoi = initial_point.buffer(3000)  # 30-meter buffer around the central point

# Define the date range
start_date = '2024-05-22'
end_date = '2024-05-23'



In [16]:
# Load Sentinel-2 image collection
collection = (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
              .filterBounds(aoi)
              .filterDate(start_date, end_date))
# Select bands and create a median composite
image = collection.select(['B4', 'B3', 'B2']).first()

In [23]:
# Use geemap to create a map and add the image
Map = geemap.Map()
Map.centerObject(initial_point, 15)  # Center the map around the point with zoom level 10
Map.addLayer(image, {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 3000}, 'Sentinel-2 RGB')
Map.addLayerControl()  # This will add a layer control panel to the map
Map

Map(center=[-43.99665917291535, 170.88861830313817], controls=(WidgetControl(options=['position', 'transparent…

In [22]:
# # Export the image to a local file
# geemap.ee_export_image_to_drive(
#     image=image,
#     description='Sentinel2_RGB',
#     folder='EarthEngineImages',
#     fileNamePrefix='Sentinel2_RGB_TEST',
#     scale=10,  # Sentinel-2 resolution in meters
#     region=aoi,
#     fileFormat='GeoTIFF',
#     maxPixels=1e13
# )

In [27]:


# Use geemap to create a map and add the image
geemap.ee_export_image(
    ee_object=image,
    filename='Sentinel2_RGB_local.tif',
    scale=10,  # Sentinel-2 resolution in meters
    region=aoi,
    file_per_band=False
)

Generating URL ...
Please wait ...
Data downloaded to c:\Users\hzh166\OneDrive - University of Canterbury\DATA690_MSc_Thesis_Hao_37041529\CodeWorks\69660_New\cleanup\Sentinel2_RGB_local.tif


In [28]:
from PIL import Image
import numpy as np
import rasterio

# Open the exported GeoTIFF
with rasterio.open('Sentinel2_RGB_local.tif') as src:
    img_array = src.read([1, 2, 3])  # Read the RGB bands
    img_array = np.transpose(img_array, (1, 2, 0))  # Transpose to (height, width, channels)

    # Normalize the values to the range 0-255 for JPEG
    img_array = img_array.astype(np.float32)
    img_array -= img_array.min()
    img_array /= img_array.max()
    img_array *= 255
    img_array = img_array.astype(np.uint8)

# Convert to JPEG and save
img = Image.fromarray(img_array)
img.save('Sentinel2_RGB_local.jpg')